## Steps to take
1. Generate node family
2. Set up ODE matrix vector 2D
3. Set up ODE boundary conditions 2D
4. Sparse solve ??
5. Derivative operator 2D 

In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import lu_factor, lu_solve


In [2]:
#Grid generator
#Here we will create a space time peridynamic grid

num_node_x = 41 #Number of nodes in the x dimension
num_node_y = 41 #Number of nodes in the y dimension
n_order = 2
dxy = 0.025 #Spacing between the nodes
horiz = 3 #Defining our Peridynamic horizon
delta_mag =(n_order+1)*dxy

symmetric = True #This flag allows us to define time as asymmetric
vol = dxy*dxy
if num_node_x % 2 == 0 & num_node_y % 2 == 0:
    x = np.arange(dxy/2, 1, dxy)
    y = np.arange(dxy/2, 1, dxy)
else:
    x = np.arange(0, 1+dxy, dxy)
    y = np.arange(0, 1+dxy, dxy)

indexing = 'xy'

xx, yy = np.meshgrid(x, y, indexing=indexing)
xx = xx.reshape(-1,1)
yy = yy.reshape(-1,1)

grid = np.concatenate((xx, yy), axis=-1)

In [3]:
def Weights2D(xsi):
    xsi_mag = np.linalg.norm(xsi, axis=1)
    #delta_mag = 0.2015e-1
    weights = []
    for i in range(xsi_mag.size):
        if(xsi_mag[i]==0):
            weights.append(-1)
        else:
            weights.append(np.exp(-4*(xsi_mag[i]/delta_mag)**2))
    return weights

def GenAmat(exi):
    weights = Weights2D(exi)
    Amat = np.zeros([6, 6])
    for i, xsi in enumerate(exi):
        xsi_mag = np.linalg.norm(xsi)
        if xsi_mag<=2*delta_mag: #This ensures that points are within the horizon
            PVec = np.array([1.0, xsi[0], xsi[1], xsi[0]**2, xsi[0]*xsi[1], xsi[1]**2])
            PVecMat = np.outer(PVec, PVec)
            if weights[i]==-1:
                Amat +=np.zeros([6, 6])
            else:
                Amat += PVecMat*weights[i]*vol
    return Amat

def InvGFunc(exi, amat):
    weights = Weights2D(exi)
    GFS = np.zeros([6, 1])
    for i, xsi in enumerate(exi):
        xsi_mag = np.linalg.norm(xsi)
        if xsi_mag<=delta_mag: #This ensures that points are within the horizon
            PVec = np.array([1.0, xsi[0], xsi[1], xsi[0]**2, xsi[0]*xsi[1], xsi[1]**2])
            PVecMat = np.matmul(PVec, amat)
            if weights[i]==-1:
                GFS += np.zeros([6, 1])
                #aux=0
            else:
                GFS += PVecMat*weights[i]*vol
                #print(PVecMat*weights[i]*vol)
                #print(len(exi))
                #print(weights[i])
                print(GFS)
                a = input('').split(" ")[0]
                
    return GFS

In [4]:
#Family members
#After creating mesh I will construct a numpy array for the family members of each node


IX, IY = np.unravel_index(99, (num_node_x, num_node_y))
xref = np.linspace(-horiz, horiz, horiz*2+1, dtype=int)

if symmetric:
    yref = np.linspace(-horiz, horiz, horiz*2+1, dtype=int)
else:
    yref = np.linspace(0, horiz, horiz+1, dtype=int)

bmat = np.eye(6)
bmat[3, 3] = 2.0
bmat[5, 5] = 2.0    
for I, crdI in enumerate(grid):
    IX, IY = np.unravel_index(I, (num_node_x, num_node_y))
    xref = np.linspace(-horiz, horiz, horiz*2+1, dtype=int)
    if np.any(xref+IX<0):
        xref -= np.min(xref+IX)      
    elif np.any(xref+IX>=num_node_x):     
        xref -= np.max(xref+IX)-num_node_x+1
        
    yref = np.linspace(-horiz, horiz, horiz*2+1, dtype=int)
    if np.any(yref+IY<0):   
        yref -= np.min(yref+IY)
    elif np.any(yref+IY>=num_node_y):  
        yref -= np.max(yref+IY)-num_node_y+1
                   
    xref, yref = np.meshgrid(xref, yref, indexing=indexing)
    exi_ref = np.concatenate((xref.reshape((-1,1)), yref.reshape((-1,1))), axis=-1)
    distI = np.sqrt(np.sum(exi_ref**2, axis=-1))
    orderI = np.argsort(distI, axis=0)
    exi_ref = exi_ref[orderI, :]
    exiI = exi_ref
    ptsI = np.array([[IX, IY]]) + exi_ref
    ptsI = np.ravel_multi_index(ptsI.T, (num_node_x, num_node_y))
    
    if not symmetric:
        mask = np.where(ptsI>=(ptsI[0]+num_node_x)//num_node_x*num_node_x)
        ptsI = np.delete(ptsI, mask)
    aux = np.zeros(num_node_x*num_node_y) 
    aux[ptsI]=1
    
    #Family members of current node
    family_members = np.where(aux == 1) 

    #Calculation of exis of current node
    current_point_coords = grid[I] 
    exis = np.longdouble(grid[family_members]-current_point_coords)
    
    #Calculation of A matrix of current node
    Amat = GenAmat(exis)
    #Here I would have to apply constraints on Amat
    
    #amat = np.linalg.solve(Amat, bmat)
    
    lu, piv = lu_factor(Amat)
    amat = lu_solve((lu, piv), bmat)
    #GFMAT = InvGFunc(exis, amat)
    #print(GFMAT)
    #GFMAT = np.zeros([6, len(exiI)])
    #for i, xi in enumerate(exiI):
        #GFMAT[:, i] = InvGFunc(xi, deltaI, amat)*VOL[horI[i]]
    #GFMAT[:, i] = InvGFunc(exis, deltaI, amat)*VOL[horI[i]]
    print(amat)
    a = input('').split(" ")[0]
    #Here the fortran code says to apply constraints on Amat
    
    #Now I have to calculate A matrices for each point on the Peridynamic Mesh
   


[[ 1.58866333e+04 -6.54510787e+05 -6.54510787e+05  1.11558599e+07
   1.09371347e+07  1.11558599e+07]
 [-6.54510787e+05  3.44105198e+07  2.54986064e+07 -7.33286598e+08
  -4.79950665e+08 -4.17556816e+08]
 [-6.54510787e+05  2.54986064e+07  3.44105198e+07 -4.17556816e+08
  -4.79950665e+08 -7.33286598e+08]
 [ 5.57792994e+06 -3.66643299e+08 -2.08778408e+08  1.03221931e+10
   3.48880966e+09  3.55853576e+09]
 [ 1.09371347e+07 -4.79950665e+08 -4.79950665e+08  6.97761931e+09
   1.18186676e+10  6.97761931e+09]
 [ 5.57792994e+06 -2.08778408e+08 -3.66643299e+08  3.55853576e+09
   3.48880966e+09  1.03221931e+10]]

[[ 1.76891436e+03  6.03732223e+03 -7.22932807e+04 -1.64307977e+06
   2.60778483e+05  1.59142892e+06]
 [ 6.03732223e+03  1.41977336e+06 -5.69465996e+03 -3.44601494e+07
  -3.04153611e+07  2.85158811e+06]
 [-7.22932807e+04 -5.69465996e+03  8.18542008e+06  3.52584163e+07
  -1.43316534e+07 -2.67941345e+08]
 [-8.21539886e+05 -1.72300747e+07  1.76292081e+07  2.07813467e+09
  -6.35901668e+07 -3.88

KeyboardInterrupt: Interrupted by user

In [ ]:
family_members

In [ ]:
bmat = np.eye(6)
bmat[3, 3] = 2.0
bmat[5, 5] = 2.0
#amat = inv(h)
h

In [ ]:
Amat = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
#bmat = np.eye(6)
#bmat[3, 3] = 2.0
#bmat[5, 5] = 2.0
bmat = np.array([1,1,1])
amat = np.linalg.solve(Amat, bmat)
print(amat)

In [ ]:
def ExpInfFunc(xi, Delta):
    wx = np.exp(-2.0 * (xi[0] / Delta[0])**2)
    wy = np.exp(-2.0 * (xi[1] / Delta[1])**2)
    return wx*wy

def GenAmat(xi, Delta):
    w = ExpInfFunc(xi, Delta)
    Avec = np.array([1.0, xi[0], xi[1], xi[0]**2, xi[0]*xi[1], xi[1]**2])
    Amat = np.outer(Avec, Avec) * w
    return Amat

def InvGFunc(xi, Delta, amat):
    w = ExpInfFunc(xi, Delta)
    Avec = np.array([1.0, xi[0], xi[1], xi[0]**2, xi[0]*xi[1], xi[1]**2])
    GFS = np.matmul(Avec, amat) * w
    return GFS

def calcGs():
    for I, crdI in enumerate(grid):
        print(I)
        print
        #delta
    return 0

calcGs()
#def calcGs(num_node, num_feature, horiz):

#    bmat = np.eye(6)
#    bmat[3, 3] = 2.0
#    bmat[5, 5] = 2.0

#    for num_nn_feature in [num_feature]:
#        horiz = ((np.sqrt(num_nn_feature)-1)/2).astype('int')

#        BGF00, BGF10, BGF01, BGF20, BGF11, BGF02 = [], [], [], [], [], []
#        for I, crdI in enumerate(CRD):
#            horI = HOR[I, :num_nn_feature]
#            exiI = np.concatenate((XIX[I, :num_nn_feature].reshape((-1,1)),XIY[I, :num_nn_feature].reshape((-1,1))), axis=1)
#            if pd_mesh_type=='ghost':
#                raise TypeError('no ghost')
#            else:
#                deltaI = np.max(np.abs(exiI), axis=0)
#            Amat = np.zeros([6, 6])
#            for i, xi in enumerate(exiI):
#                Amat += GenAmat(xi, deltaI)*VOL[horI[i]]
#            amat = np.linalg.solve(Amat, bmat)

#            GFMAT = np.zeros([6, len(exiI)])
#            for i, xi in enumerate(exiI):
#                GFMAT[:, i] = InvGFunc(xi, deltaI, amat)*VOL[horI[i]]


#            BGF00.append(GFMAT[0:1, :])
#            BGF10.append(GFMAT[1:2, :])
#            BGF01.append(GFMAT[2:3, :])
#            BGF20.append(GFMAT[3:4, :])
#            BGF11.append(GFMAT[4:5, :])
#            BGF02.append(GFMAT[5:6, :])

#        BGF00 = np.concatenate(BGF00)
#        BGF10 = np.concatenate(BGF10)
#        BGF01 = np.concatenate(BGF01)
#        BGF20 = np.concatenate(BGF20)
#        BGF11 = np.concatenate(BGF11)
#        BGF02 = np.concatenate(BGF02)

#        Famil = HOR[:, :num_nn_feature]
#        GF00 = BGF00[:, :num_nn_feature]
#        GF10 = BGF10[:, :num_nn_feature]
#        GF01 = BGF01[:, :num_nn_feature]
#        GF20 = BGF20[:, :num_nn_feature]
#        GF11 = BGF11[:, :num_nn_feature]
#        GF02 = BGF02[:, :num_nn_feature]

#    return GF10, GF01, Famil

In [ ]:
print(family_members[500])
print(exis[500])

In [ ]:
exi_x = []; exi_t = [];
indices = family_members[100][0]
coords_x = grid[indices][13][0]
coords_y = grid[indices][13][1]
print(indices)
print(indices.size)
print(coords_x)
print(coords_y)

In [ ]:
#family_members = np.array(pts)
#print(family_members)
indices = family_members[200][0]
aux = grid[indices]
print(indices)
print(aux)
#exi_x = []
#exi_t = []
#family = family_members[50][0]
#print(family)
#for node in range(len(family_members)):
#    family = family_members[node][0]
#    coords = grid[family]
#    current_x_coord = coords[node][0]
#    current_y_coord = coords[node][1]
#    print(node)
#    print(family)
    #for i in family:
        
#    print(coords)
#    print(current_x_coord)
#    print(current_y_coord)
#    a = input('').split(" ")[0]
#for family in
#print(len(family_members))

In [ ]:
family_members[600]


In [ ]:
#Checking to see if the correct indices are selected
indices = np.where(family_members[0] == 1) 
print(indices)

In [ ]:
aux = grid[indices]

In [ ]:
print(aux)
print(aux[1][0])


In [ ]:
#What I have to calculate now are the exi_x and exi_t matrices